In [22]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

species = pd.read_csv('species_info.csv')
observations = pd.read_csv('observations.csv')

print(species.head(20))
print(observations.head(20))



   category                scientific_name  \
0    Mammal  Clethrionomys gapperi gapperi   
1    Mammal                      Bos bison   
2    Mammal                     Bos taurus   
3    Mammal                     Ovis aries   
4    Mammal                 Cervus elaphus   
5    Mammal         Odocoileus virginianus   
6    Mammal                     Sus scrofa   
7    Mammal                  Canis latrans   
8    Mammal                    Canis lupus   
9    Mammal                    Canis rufus   
10   Mammal       Urocyon cinereoargenteus   
11   Mammal                   Vulpes fulva   
12   Mammal                  Vulpes vulpes   
13   Mammal                 Felis concolor   
14   Mammal               Felis silvestris   
15   Mammal                     Lynx rufus   
16   Mammal                  Puma concolor   
17   Mammal              Mephitis mephitis   
18   Mammal             Spilogale putorius   
19   Mammal              Lontra canadensis   

                                 

In [21]:
# Explore the data

print(f"species shape: {species.shape}")
print(f"observations shape: {observations.shape}")


print(f"species columns: {species.columns}")
print(f"observations columns: {observations.columns}")

print(f"number of species:{species.scientific_name.nunique()}")
print(f"number of categories: {species.category.nunique()}")
print(f"categories: {species.category.unique()}")
print(species.groupby("category").size())

print(f"number of conservation statuses:{species.conservation_status.nunique()}")
print(f"conservation statuses:{species.conservation_status.unique()}")

print(f"na values:{species.conservation_status.isna().sum()}")

print(species.groupby("conservation_status").size())

print(f"number of parks: {observations.park_name.nunique()}")
print(f"parks: {observations.park_name.unique()}")

print(f"number of observations:{observations.observations.sum()}")



species shape: (5824, 4)
observations shape: (23296, 3)
species columns: Index(['category', 'scientific_name', 'common_names', 'conservation_status'], dtype='object')
observations columns: Index(['scientific_name', 'park_name', 'observations'], dtype='object')
number of species:5541
number of categories: 7
categories: ['Mammal' 'Bird' 'Reptile' 'Amphibian' 'Fish' 'Vascular Plant'
 'Nonvascular Plant']
category
Amphibian              80
Bird                  521
Fish                  127
Mammal                214
Nonvascular Plant     333
Reptile                79
Vascular Plant       4470
dtype: int64
number of conservation statuses:4
conservation statuses:[nan 'Species of Concern' 'Endangered' 'Threatened' 'In Recovery']
na values:5633
conservation_status
Endangered             16
In Recovery             4
Species of Concern    161
Threatened             10
dtype: int64
number of parks: 4
parks: ['Great Smoky Mountains National Park' 'Yosemite National Park'
 'Bryce National Park' 'Ye